# Parsing PDFs 

Pull in all of the PDF files and create objects for the text inside each one. 

This also probably should include a spell check

[One of the sources I am using for the topic modeling](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/)
[This is a good post on Lemmatizing in python](https://www.machinelearningplus.com/nlp/lemmatization-examples-python/)

In [1]:
import string 

In [2]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [3]:
import PyPDF2 
from glob import glob

pdfs = glob('../pdfs/*.pdf') 

In [4]:
pdfs

['../pdfs/Cruz - 2010 - Chapter Six. Expanding The View The Challenges Of.pdf',
 '../pdfs/Cruz - 2010 - Index.pdf',
 '../pdfs/Frederiks and Nagy - 2016 - Religion, migration, and identity methodological .pdf',
 '../pdfs/Cruz - 2010 - Introduction.pdf',
 '../pdfs/Cruz - 2010 - Chapter One. Geographies Of Domestication Mapping.pdf',
 '../pdfs/cruz2010.pdf',
 '../pdfs/Cruz - 2010 - Chapter Two. Frontiers Of Struggle Negotiating Fi.pdf',
 '../pdfs/Cruz - 2010 - Conclusion.pdf',
 '../pdfs/Cruz - 2010 - Chapter Four. Exploring Theological Markers Delor.pdf',
 '../pdfs/Cruz - 2010 - An intercultural theology of migration pilgrims i.pdf',
 "../pdfs/Nguyen and Prior - 2014 - God's people on the move biblical and global pers.pdf",
 '../pdfs/Cruz - 2010 - Chapter Three. Expanding The Boundaries Theologic.pdf',
 '../pdfs/Gods People on the Move 2.pdf',
 '../pdfs/Cruz - 2010 - Chapter Five. A Different Cartography Mapping The.pdf',
 '../pdfs/Izuzquiza - 2011 - Breaking bread notes for a political t

# Preparing Texts 

## Lemmatizing and cleaning 

In [5]:
import string
import nltk 
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as STOPWORDS 

PUNCDIG_TRANSLATOR = str.maketrans('', '', string.punctuation+string.digits)

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sgoodwin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/sgoodwin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
def text_clean(text):
    clean_list = []
    words = nltk.word_tokenize(text)
    for w in words:
        if w not in STOPWORDS:
            w = w.translate(PUNCDIG_TRANSLATOR)
            clean_list.append(lemmatizer.lemmatize(w))
    return clean_list

In [7]:
text_clean("The striped bats are hanging on their feet3 for best.".lower())

['striped', 'bat', 'hanging', 'foot', 'best', '']

## Extracting Texts

In [8]:
pdf = open(pdfs[0], 'rb')
pdf_obj = PyPDF2.PdfFileReader(pdf)


In [9]:
print('No. of pages: {}'.format(pdf_obj.numPages))

No. of pages: 35


In [10]:
def pdf_extractor(pdf, corpus_list, text_list):
    '''Extract the text of pdfs and return a dictionary with
    the file name as a key, and the value being a list of the pages
    and the containing texts
    '''
    pdf_file_obj = open(pdf, 'rb')
    pdf_obj = PyPDF2.PdfFileReader(pdf_file_obj)
    for pn in range(0,pdf_obj.numPages):
        page = pdf_obj.getPage(pn)
        text = page.extractText().lower()
        cleaned_list = text_clean(text)
        corpus_list.append(cleaned_list)
        # corpus_list.append(page.extractText())
        text_list.append((pdf, pn))
        # if you want to create a dictionary
        # text_dict.setdefault(pdf, []).append(page.extractText())
    pdf_file_obj.close()
    return corpus_list, text_list

In [11]:
corpus_list = []
text_list = []

for pdf in pdfs:
    corpus_list, text_list = pdf_extractor(pdf, corpus_list, text_list)

# Creating LDA Model 



In [12]:
from gensim import corpora 
from gensim.models.ldamodel import LdaModel 

In [13]:
def prepare_topic_model(corpus_list):
    corpus_dict = corpora.Dictionary(corpus_list)
    corpus_dict.filter_extremes(no_below=100, no_above=0.5)
    corpus = [corpus_dict.doc2bow(text) for text in corpus_list]
    lda_model = LdaModel(corpus=corpus, 
                        id2word=corpus_dict, num_topics=25,
                        random_state=100, update_every=1,
                        chunksize=100, passes=50,
                        alpha='symmetric', per_word_topics=True)
    return lda_model, corpus, corpus_dict

In [14]:
lda_model, corpus, corpus_dict = prepare_topic_model(corpus_list)

# Visualizing LDA Model

In [15]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
pyLDAvis.enable_notebook()
# The sort_topics=False makes the topic model numbers agree [+1] with the topic model from gensim
# Gensim's topic numbers' are zero indexed, and the vis index is 1 indexed
vis = pyLDAvis.gensim.prepare(lda_model, corpus, corpus_dict, sort_topics=False, mds='mmds')

of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.
 [_prepare.py:257]


In [43]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.141876  0.451939       1        1   3.541566
1      0.237688 -0.179055       2        1   2.255508
2     -0.309638 -0.255653       3        1   4.095959
3      0.508020  0.176455       4        1   1.035574
4      0.139424  0.432739       5        1   4.852592
5     -0.279917 -0.427460       6        1   1.704989
6     -0.167615 -0.124321       7        1   5.764232
7     -0.512703  0.111494       8        1   1.378420
8      0.046413 -0.256209       9        1   3.555053
9      0.480339 -0.206170      10        1   3.076550
10     0.310442  0.333594      11        1   3.449644
11     0.215111  0.099568      12        1  12.626311
12     0.085143 -0.470862      13        1   2.762902
13     0.074581  0.278889      14        1  19.958881
14    -0.078999 -0.437227      15        1   2.439882
15    -0.284734  0.117231      16        1   3.870130
16    -0.003352  0.035062      17        1   0.069642
17     0.447106 -0.046948      18        1   1.167811
18    -0.252002  0.007853      19        1   4.217513
19     0.167973  0.268544      20        1   9.021376
20    -0.176332  0.306838      21        1   2.913751
21    -0.337595  0.305766      22        1   3.449894
22    -0.478421 -0.119964      23        1   1.875947
23    -0.003352  0.035062      24        1   0.069642
24     0.314297 -0.437164      25        1   0.846242, topic_info=     Category       Freq          Term        Total  loglift  logprob
term                                                                 
33    Default  2532.0000             ”  2532.000000  30.0000  30.0000
84    Default   932.0000        church   932.000000  29.0000  29.0000
48    Default  1812.0000             ’  1812.000000  28.0000  28.0000
32    Default  2171.0000             “  2171.000000  27.0000  27.0000
31    Default   680.0000             –   680.000000  26.0000  26.0000
15    Default   944.0000     migration   944.000000  25.0000  25.0000
24    Default   704.0000      theology   704.000000  24.0000  24.0000
28    Default  1182.0000         woman  1182.000000  23.0000  23.0000
53    Default   929.0000       migrant   929.000000  22.0000  22.0000
11    Default   689.0000      filipino   689.000000  21.0000  21.0000
21    Default   583.0000      struggle   583.000000  20.0000  20.0000
72    Default  1016.0000           dhs  1016.000000  19.0000  19.0000
43    Default  1019.0000             s  1019.000000  18.0000  18.0000
7     Default   797.0000             e   797.000000  17.0000  17.0000
66    Default   371.0000          work   371.000000  16.0000  16.0000
29    Default   446.0000        worker   446.000000  15.0000  15.0000
9     Default   437.0000        eology   437.000000  14.0000  14.0000
8     Default   416.0000            ed   416.000000  13.0000  13.0000
45    Default   317.0000        social   317.000000  12.0000  12.0000
90    Default   241.0000             d   241.000000  11.0000  11.0000
89    Default   281.0000         right   281.000000  10.0000  10.0000
16    Default   534.0000           new   534.000000   9.0000   9.0000
51    Default   259.0000      economic   259.000000   8.0000   8.0000
63    Default   352.0000      religion   352.000000   7.0000   7.0000
55    Default   491.0000        people   491.000000   6.0000   6.0000
57    Default   395.0000     religious   395.000000   5.0000   5.0000
18    Default   357.0000    philippine   357.000000   4.0000   4.0000
35    Default   435.0000           god   435.000000   3.0000   3.0000
6     Default   333.0000      domestic   333.000000   2.0000   2.0000
105   Default   255.0000     immigrant   255.000000   1.0000   1.0000
...       ...        ...           ...          ...      ...      ...
5     Topic25     0.0309           doe   185.345108  -3.9271  -9.3032
12    Topic25     0.0309          form   190.045227  -3.9521  -9.3032
30    Topic25     0.0309          york   191.2444

In [44]:
pyLDAvis.save_html(vis, 'PrelimTopicModel.html')

# Find the Dominant Document For Each Topic 


In [18]:
import pandas as pd

In [19]:
# this creates a pandas DataFrame that orders all of the topics and shows the dominant topic for each document
def format_topics_sent(ldamodel, corpus, texts):
    sent_topics_df = pd.DataFrame()
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: x[1], reverse=True)
        
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_topic', 'Perc_Contrib', 'Topic_Keywords']
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return sent_topics_df

## Exploring the Dominant Topic Models

In [20]:
def format_topics_sent_gen(ldamodel, corpus, texts):
    for i, row in enumerate(ldamodel[corpus]):
        yield row

In [21]:
genny = format_topics_sent_gen(lda_model, corpus, corpus_list)

In [22]:
row = next(genny)

In [23]:
len(row)

3

In [42]:
lda_model.show_topic(13)

[('”', 0.2966608),
 ('“', 0.2558419),
 ('’', 0.038733605),
 ('–', 0.03453399),
 ('vol', 0.03118087),
 ('e', 0.030572623),
 ('dhs', 0.028041013),
 ('s', 0.021423323),
 ('tulud', 0.015217608),
 ('cruz', 0.014499779)]

In [25]:
sent_topics_df = format_topics_sent(lda_model, corpus, text_list)

In [26]:
grpd_df = sent_topics_df.groupby('Dominant_topic')

In [27]:
# This code creates a pandas DataFrame that shows which document is exemplified by which topic
sent_topics_df = pd.DataFrame()

for i, grp in grpd_df:
    sent_topics_df = pd.concat([sent_topics_df, grp.sort_values(['Perc_Contrib'], ascending=[0]).head(1)], axis=0)

sent_topics_df.reset_index(drop=True, inplace=True)
sent_topics_df.columns = ['Topic_Num', 'Topic_Perc_Contrib', 'Keywords', 'Text']
sent_topics_df

Topic_Num  Topic_Perc_Contrib  \
0         0.0              0.5008   
1         1.0              0.3923   
2         2.0              0.5300   
3         3.0              0.7007   
4         4.0              0.4219   
5         5.0              0.3031   
6         6.0              0.8080   
7         7.0              0.4001   
8         8.0              0.4924   
9         9.0              0.3283   
10       10.0              0.8040   
11       11.0              0.8343   
12       12.0              0.4642   
13       13.0              0.8413   
14       14.0              0.5705   
15       15.0              0.4450   
16       17.0              0.4085   
17       18.0              0.6731   
18       19.0              0.5663   
19       20.0              0.5590   
20       21.0              0.4630   
21       22.0              0.4607   
22       24.0              0.9262   

                                             Keywords  \
0   migration, context, migrant, study, experience...   
1   immigrant, american, case, service, sense, gro...   
2   new, ed, press, york, study, mission, book, ch...   
3   –, people, religion, dhs, order, come, life, e...   
4   ’, s, god, experience, e, challenge, g, cruz, ...   
5   economic, family, home, role, migration, er, t...   
6   nagy, dorottya, frederiks, martha, christian, ...   
7   right, power, human, political, world, come, s...   
8   religion, identity, religious, culture, cultur...   
9   struggle, filipino, philippine, eology, book, ...   
10  eology, e, feminist, cruz, tulud, g, woman, po...   
11  dhs, hong, kong, ’, filipino, hk, domestic, wo...   
12  church, christian, role, context, american, ti...   
13            ”, “, ’, –, vol, e, dhs, s, tulud, cruz   
14  migrant, worker, country, domestic, work, soci...   
15  people, place, fact, example, make, situation,...   
16  work, study, form, relation, place, question, ...   
17  life, god, word, people, way, mean, reality, t...   
18  woman, ’, s, experience, particularly, oppress...   
19  community, group, religious, migrant, faith, c...   
20  theology, theological, context, way, new, ques...   
21  social, cultural, political, society, reality,...   
22  d, c, come, dhs, life, order, case, like, hk, ...   

                                                 Text  
0   (../pdfs/Frederiks and Nagy - 2016 - Religion,...  
1   (../pdfs/Frederiks and Nagy - 2016 - Religion,...  
2   (../pdfs/Frederiks and Nagy - 2016 - Religion,...  
3                (../pdfs/Cruz - 2010 - Index.pdf, 2)  
4   (../pdfs/Cruz - 2010 - Chapter Four. Exploring...  
5   (../pdfs/Frederiks and Nagy - 2016 - Religion,...  
6   (../pdfs/Frederiks and Nagy - 2016 - Religion,...  
7   (../pdfs/Frederiks and Nagy - 2016 - Religion,...  
8   (../pdfs/Cruz - 2010 - Chapter Six. Expanding ...  
9   (../pdfs/Cruz - 2010 - Preliminary Material.pd...  
10  (../pdfs/Cruz - 2010 - Preliminary Material.pd...  
11  (../pdfs/Cruz - 2010 - Chapter One. Geographie...  
12  (../pdfs/Frederiks and Nagy - 2016 - Religion,...  
13  (../pdfs/Cruz - 2010 - Chapter One. Geographie...  
14  (../pdfs/Frederiks and Nagy - 2016 - Religion,...  
15  (../pdfs/Izuzquiza - 2011 - Breaking bread not...  
16  (../pdfs/Frederiks and Nagy - 2016 - Religion,...  
17  (../pdfs/Izuzquiza - 2011 - Breaking bread not...  
18  (../pdfs/Cruz - 2010 - Chapter Four. Exploring...  
19  (../pdfs/Frederiks and Nagy - 2016 - Religion,...  
20  (../pdfs/Frederiks and Nagy - 2016 - Religion,...  
21  (../pdfs/Izuzquiza - 2011 - Breaking bread not...  
22        (../pdfs/Gods People on the Move 2.pdf, 14)

In [28]:
sent_topics_df.iloc[0]['Text']

('../pdfs/Frederiks and Nagy - 2016 - Religion, migration, and identity methodological .pdf',
 63)

In [29]:
lda_model.show_topic(0)

[('migration', 0.5920657),
 ('context', 0.12714523),
 ('migrant', 0.0677818),
 ('study', 0.05909739),
 ('experience', 0.037804633),
 ('tion', 0.024900865),
 ('community', 0.021318907),
 ('challenge', 0.017123718),
 ('human', 0.014246021),
 ('culture', 0.012124161)]